<a href="https://colab.research.google.com/github/ShihaoCui/Mydata_1/blob/main/THM%20Inversion/SolidsPINNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>